# Detekcija i klasifikacija zvuka; Matea Lukić

Koristit ću YAMNet. To je duboka mreža koja predviđa 521 audio događaj [klase] (https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/yamnet_class_map.csv) iz [korpusa AudioSet-YouTube] (http : //g.co/audioset)  na kojem je mreža trenirana.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/STROJNO

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

Učitavamo model iz TensorFlow Hub-a.

Više o modelu se može naći na sljedećoj poveznici :  [url_modela](https://tfhub.dev/google/yamnet/1)

In [ ]:
# Ucitavanje modela
model = hub.load('https://tfhub.dev/google/yamnet/1')

Datoteka s labelama učitat će se iz modelovih assets-a i prisutna je u `model.class_map_path ()`.

Učitat ću ju u varijablu :  `class_names`.



In [ ]:
# Pronalazim naziv klase s najboljom ocjenom.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_names = []
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row['display_name'])

  return class_names

class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)

Dodaja metode za provjeru i pretvorbu učitanog zvuka u odgovarajući sample_rate (16K). Treba biti u formatu .wav 16kHz, mono.

In [ ]:
def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):  ### 16 kHz
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

## Download i priprema zvučne datoteke

Tu ćemo skinuti i preslušati .wav audio datoteku koju sam pripremila.

Napomena: Očekivana audio datoteka trebala bi biti mono wav datoteka sa brzinom uzorkovanja od 16 kHz.

Stoga, svoj audio zapis u kojem sam rekla riječ "strojno" ubacujem u konverter G711 na linku : https://g711.org/ i biram ::: 
" High Definition WAV (16Khz, Mono, 16-Bit PCM) "

Također ću učitati još neke zapise za demo, 
Jedan je mjaukanje mačke, a drugi je glazbeni isječak koji sam skinula sa sljedećeg linka : : : http://www.music.helsinki.fi/tmt/opetus/uusmedia/esim/index-e.html pod nazivom "  a2002011001-e02-16kHz.wav  " te provukla kroz spomenuti konverter. 

In [ ]:
!curl -O https://storage.googleapis.com/audioset/miaow_16k.wav # link do zvuka mjaukanja mačke

In [ ]:
!curl -O https://drive.google.com/file/d/1tSNonXTEfI9Lvyky1VXiAxGvqk2w59eX/view?usp=sharing  # link do zapisa glazbe

In [ ]:
!curl -0 https://drive.google.com/file/d/1u9quFdfEjaviTJ7pjXctnUsP9DsgYzGY/view?usp=sharing   ### link do zvucne datoteke gdje ja govorim "strojno učenje"

In [ ]:
wav_file_name = 'Govor_strojno_ucenje01.wav'
#wav_file_name = 'orkestar33.wav'
#wav_file_name = 'miaow_16k.wav'

sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)

# Show some basic information about the audio.
duration = len(wav_data)/sample_rate
print(f'Sample rate: {sample_rate} Hz')
print(f'Total duration: {duration:.2f}s')
print(f'Size of the input: {len(wav_data)}')

# Listening to the wav file.
Audio(wav_data, rate=sample_rate)

Wav datoteku treba normalizirati na vrijednosti u rangu`[-1.0, 1.0]` (kako je navedeno u [dokumentaciji](https://tfhub.dev/google/yamnet/1)).

In [ ]:
waveform = wav_data / tf.int16.max

## Izvršenje modela

Koristeći već pripremljene podatke, samo treba pozvati model i dobiju se: ocjene, ugrađivanje i spektrogram.

Score je glavni rezultat koji se koristi.
Spektrogram sluzi za vizualizaciju.

In [ ]:
# Izvršavanje modela; provjera outputa
scores, embeddings, spectrogram = model(waveform)

In [ ]:
scores_np = scores.numpy()
spectrogram_np = spectrogram.numpy()
infered_class = class_names[scores_np.mean(axis=0).argmax()]
print(f'Glavni prepoznati zvuk je : {infered_class}')

## Vizualizacija


YAMNet također vraća neke dodatne informacije koji se mogu koristiti za vizualizaciju.
Dobiveni su  : valni oblik, spektrogram i zaključene glavne klase.

In [ ]:
plt.figure(figsize=(10, 6))

# Plot the waveform.
plt.subplot(3, 1, 1)
plt.plot(waveform)
plt.xlim([0, len(waveform)])

# Plot the log-mel spectrogram (returned by the model).
plt.subplot(3, 1, 2)
plt.imshow(spectrogram_np.T, aspect='auto', interpolation='nearest', origin='lower')

# Plot and label the model output scores for the top-scoring classes.
mean_scores = np.mean(scores, axis=0)
top_n = 10
top_class_indices = np.argsort(mean_scores)[::-1][:top_n]
plt.subplot(3, 1, 3)
plt.imshow(scores_np[:, top_class_indices].T, aspect='auto', interpolation='nearest', cmap='gray_r')

# patch_padding = (PATCH_WINDOW_SECONDS / 2) / PATCH_HOP_SECONDS
# values from the model documentation
patch_padding = (0.025 / 2) / 0.01
plt.xlim([-patch_padding-0.5, scores.shape[0] + patch_padding-0.5])
# Label the top_N classes.
yticks = range(0, top_n, 1)
plt.yticks(yticks, [class_names[top_class_indices[x]] for x in yticks])
_ = plt.ylim(-0.5 + np.array([top_n, 0]))

## DRUGI DIO - obrada audio podataka

Priprema i povećanje audio podataka

## Pregled

Jedan od najvećih izazova u automatskom prepoznavanju govora je priprema i povećanje audio podataka. Analiza audio podataka može biti u vremenskoj ili frekvencijskoj domeni, što dodaje dodatni kompleks u usporedbi s drugim izvorima podataka, poput slika.

Kao dio ekosustava TensorFlow, paket `tensorflow-io` pruža nekoliko korisnih API-ja povezanih sa zvukom koji pomažu u olakšavanju pripreme i povećanja audio podataka.

## Postavke


### Instalacija potrebnih paketa

In [ ]:
!pip install tensorflow-io

## Korištenje

### Čitanje audio datoteke

U TensorFlow-IO-u, klasa `tfio.audio.AudioIOTensor` omogućuje čitanje audio datoteke u učitani` IOTensor`:

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

#audio = tfio.audio.AudioIOTensor('orkestar33.wav')
audio = tfio.audio.AudioIOTensor('Govor_strojno_ucenje01.wav') 

print(audio)



`AudioIOTensor` je lazy-loaded, tako da su u početku prikazani samo oblik, vrsta i brzina uzorkovanja. Oblik `AudioIOTensor` predstavljen je kao` [uzorci, kanali] `, što znači da je učitani audio isječak koji ste mono kanal s` 28979` uzorcima u `int16`.

Sadržaj audio isječka čitat će se samo po potrebi, bilo pretvaranjem `AudioIOTensor` u` Tensor` kroz `to_tensor ()` ili rezanjem. Rezanje je posebno korisno kada je potreban samo mali dio velikog audio isječka:

In [ ]:
audio_slice = audio[5:]

#  uklanjamo zadnju dimenziju
audio_tensor = tf.squeeze(audio_slice, axis=[-1])

print(audio_tensor)


Audio može biti pokrenut i preslušan kroz:

In [ ]:
from IPython.display import Audio

Audio(audio_tensor.numpy(), rate=audio.rate.numpy())

Pogodnije je pretvoriti tenzor u float brojeve i prikazati audio isječak na grafikonu:

In [ ]:
import matplotlib.pyplot as plt


tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

plt.figure()
plt.plot(tensor.numpy())

### Smanjivanje šumova

Ponekad ima smisla smanjiti šumove iz zvuka, što se može učiniti putem API `tfio.experimental.audio.trim`. Iz API-ja je vraćen par položaja `[start, stop]` segmenta:

In [ ]:
position = tfio.experimental.audio.trim(tensor, axis=0, epsilon=0.1)
print(position)

start = position[0]
stop = position[1]
print(start, stop)

processed = tensor[start:stop]

plt.figure()
plt.plot(processed.numpy())

### Fade In i Fade Out

Jedna korisna tehnika audio inžinjeringa je *fade* koja postupno povećava ili smanjuje audio signale. To se može učiniti putem `tfio.experimental.audio.fade`. `tfio.experimental.audio.fade` podržava različite oblike *fade* -a  kao što su" linearni "," logaritamski "ili" eksponencijalni ":

In [ ]:
fade = tfio.experimental.audio.fade(
    processed, fade_in=1000, fade_out=2000, mode="logarithmic")

plt.figure()
plt.plot(fade.numpy())

### Spektrogram

Napredna obrada zvuka često djeluje na promjene frekvencije tijekom vremena. U `tensorflow-io` valni oblik može se pretvoriti u spektrogram putem` tfio.experimental.audio.spectrogram`:

In [ ]:
#Konvertiranje u spektogram
spectrogram = tfio.experimental.audio.spectrogram(
    fade, nfft=512, window=512, stride=256)

plt.figure()
plt.imshow(tf.math.log(spectrogram).numpy())

Moguće su i dodatne transformacije u različite ljestvice:

In [ ]:
# Konvertiraj u  mel-spektogram
mel_spectrogram = tfio.experimental.audio.melscale(
    spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)


plt.figure()
plt.imshow(tf.math.log(mel_spectrogram).numpy())

# Konvertiraj u db scale  mel-spektogram
dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
    mel_spectrogram, top_db=80)

plt.figure()
plt.imshow(dbscale_mel_spectrogram.numpy())

### SpecAugment

Uz gore spomenute API-je za pripremu i proširivanje podataka, paket `tensorflow-io` također nudi napredne augmentacije spektrograma, ponajviše maskiranje frekvencije i vremena o kojima se raspravlja u [SpecAugment: Jednostavna metoda povećanja podataka za automatsko prepoznavanje govora (Park i sur., 2019)] (https://arxiv.org/pdf/1904.08779.pdf).

#### Maskiranje frekvencije

U maskiranju frekvencije, maskirani su frekvencijski kanali `[f0, f0 + f)` gdje je `f 'odabran iz ujednačene raspodjele od' 0 'do parametra frekvencijske maske` F', a `f0 'je odabran od` (0 , ν - f) `gdje je` ν` broj frekvencijskih kanala.

In [ ]:
# Freq masking
freq_mask = tfio.experimental.audio.freq_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(freq_mask.numpy())

#### Maskiranje vremena

U maskiranju vremena maskirani su `t` uzastopni vremenski koraci` [t0, t0 + t) `gdje je` t` odabran iz jednolike raspodjele od `0` do parametra vremenske maske` T`, a odabran je `t0` od `[0, τ - t)` gdje su 'τ' vremenski koraci.Ž

In [ ]:
# Time masking
time_mask = tfio.experimental.audio.time_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(time_mask.numpy())